# Preliminary Data and Functions

In [1]:
import numpy as np
import re
from random import randint

#Importing Data
wordsList = np.load('Numpy Arrays/wordsList.npy')
wordsList = wordsList.tolist()
wordsList = [word.decode('UTF-8') for word in wordsList]
wordVectors = np.load('Numpy Arrays/wordVectors.npy')
#text = np.load('Numpy Arrays/text.npy')
the_labels = np.load('Numpy Arrays/labels.npy')
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

#Helper Functions
def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())
ids = np.load('Numpy Arrays/idsMatrix.npy')
def getTrainBatch():
    label = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1, 1000000)
        arr[i] = ids[num]
        label.append(the_labels[num])
    return arr, label
def getTestBatch():
    label = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1000001, the_labels.shape[0]-1)
        arr[i] = ids[num]
        label.append(the_labels[num])
    return arr, label

# BRNN Model

### Hyper Parameters

In [2]:
batchSize = 512
lstmUnits = 128
numClasses = 2
iterations = 10000
learning_rate = 0.001
maxSeqLength = 20

In [3]:
import tensorflow as tf
with tf.name_scope("Input_Data") as scope:
    labels = tf.placeholder(tf.float32, [batchSize, numClasses], name='Sentiment_Labels')
    input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength], name='Text')

with tf.name_scope("Data_Preprocessing") as scope:
    data = tf.Variable(tf.zeros([batchSize, maxSeqLength, 3]),dtype=tf.float32, name='Embed_Text')
    data = tf.nn.embedding_lookup(wordVectors,input_data)

In [4]:
with tf.name_scope("BRNN") as scope:
    lstmCell1 = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
    lstmCell1 = tf.contrib.rnn.DropoutWrapper(cell=lstmCell1, output_keep_prob=0.9)
    lstmCell2 = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
    lstmCell2 = tf.contrib.rnn.DropoutWrapper(cell=lstmCell2, output_keep_prob=0.9)
    (forward_output, backward_output), _ = tf.nn.bidirectional_dynamic_rnn(lstmCell1, lstmCell2, inputs=data, 
                                                                               dtype=tf.float32)
    outputs = tf.concat([forward_output, backward_output], axis=2)

In [5]:
with tf.name_scope("Prediction_Parameters") as scope:
    weight = tf.Variable(tf.truncated_normal([2*lstmUnits, numClasses]), name='Weights')
    bias = tf.Variable(tf.constant(0.1, shape=[numClasses]), name='bias')
    value = tf.transpose(outputs, [1, 0, 2])
    last = tf.gather(value, int(value.get_shape()[0]) - 1)
with tf.name_scope("Prediction") as scope:
    prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

In [6]:
with tf.name_scope("Accuracy") as scope:
    correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))


In [7]:
with tf.name_scope("Loss") as scope:
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=labels))
with tf.name_scope("Train") as scope:
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [8]:
import datetime
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
validation = tf.summary.scalar('Validation', accuracy)
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

NameError: name 'sess' is not defined

In [9]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())


# Training

In [10]:
for i in range(iterations+1):
        #Next Batch of reviews
    nextBatch, nextBatchLabels = getTrainBatch();
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
    #Write summary to Tensorboard
#    if (i % 5 == 0):
#        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
#        writer.add_summary(summary, i)
#        nextTestBatch, nextTestBatchLabels = getTestBatch();
#        val = sess.run(validation, {input_data: nextTestBatch, labels: nextTestBatchLabels})
#        writer.add_summary(val, i)
    if (i%500 == 0):
        print("Loss: " + str(sess.run(loss, {input_data: nextBatch, labels: nextBatchLabels})))
save_path = saver.save(sess, "BRNN/brnn.ckpt", global_step=i)
writer.close()

Loss: 0.6881615
Loss: 0.60475194
Loss: 0.5858214
Loss: 0.56983113
Loss: 0.5404841
Loss: 0.53837615
Loss: 0.53830457
Loss: 0.5426141
Loss: 0.5396477
Loss: 0.5516456
Loss: 0.5386627
Loss: 0.51551133
Loss: 0.5371313
Loss: 0.54766214
Loss: 0.5236435
Loss: 0.5276114
Loss: 0.519515
Loss: 0.5179168
Loss: 0.54132223
Loss: 0.5286012
Loss: 0.50837076


NameError: name 'writer' is not defined

In [121]:
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix
def test(sent):
    x = ['negetive', 'positive']
    sent = getSentenceMatrix(sent)
    pred = sess.run(prediction, {input_data:sent})[0]
    return(pred)
def results():
    arr = np.array([[1, 0]], dtype=np.float32)
    arr_pred = np.array([test("my life sucks")])
    for i in range(512, ids.shape[0], 512):
        arr = np.vstack((arr, the_labels[i-512:i]))
        arr_pred = np.vstack((arr_pred, sess.run(prediction, {input_data:ids[i-512:i]})))
    x = np.argmax(arr_pred, axis=1)
    y = np.argmax(arr, axis=1)
    return((np.sum(np.equal(x, y))/x.shape))

In [122]:
results()

array([0.77203105])